# Práctica 2: Procesamiento del Lenguaje Natural

__Fecha de entrega: 8 de mayo de 2023__

El objetivo de esta práctica es aplicar los conceptos teóricos vistos en clase en el módulo de PLN. La práctica consta de 2 notebooks que se entregarán simultáneamente en la tarea de entrega habilitada en el Campus  Virtual.

Lo más importante en esta práctica no es el código Python, sino el análisis de los datos y modelos que construyas y las explicaciones razonadas de cada una de las decisiones que tomes. __No se valorarán trozos de código o gráficas sin ningún tipo de contexto o explicación__.

Finalmente, recuerda establecer el parámetro `random_state` en todas las funciones que tomen decisiones aleatorias para que los resultados sean reproducibles (los resultados no varíen entre ejecuciones).

In [19]:
RANDOM_STATE = 1234

# Apartado 1: Análisis de sentimientos con word embeddings


__Número de grupo: 20__

__Nombres de los estudiantes: Alejandro Barrachina Argudo y Juan Pablo Corella Martín__

## 1) Carga del conjunto de datos

El fichero `IMBD_Dataset.csv` contiene opiniones de películas clasificadas en 2 categorías diferentes (positiva/negativa).

Este set de datos se creó utilizando el "IMDB Dataset of 50K Movie Reviews", el cual contiene 50,000 reseñas de películas con un sentimiento positivo o negativo adjunto a ellas.

Muestra un ejemplo de cada clase.

Haz un estudio del conjunto de datos. ¿qué palabras aparecen más veces?, ¿tendría sentido normalizar de alguna manera el corpus?

Crea una partición de los datos dejando el 80% para entrenamiento y el 20% restante para test usando la función `train_test_split` de sklearn.


In [20]:
# acceso a google drive

# from google.colab import drive
# drive.mount('/content/drive')

In [21]:
import pandas as pd
import numpy as np


In [22]:
# imbd_file = '/content/drive/MyDrive/IA2/p3/IMDB_Dataset.csv'
imbd_file = './IMDB_Dataset.csv'

df=pd.read_csv(imbd_file)
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [23]:
print("Positivo:\n" + df.review[0])
print("Negativo:\n" + df.review[3])

Positivo:
One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me.<br /><br />The first thing that struck me about Oz was its brutality and unflinching scenes of violence, which set in right from the word GO. Trust me, this is not a show for the faint hearted or timid. This show pulls no punches with regards to drugs, sex or violence. Its is hardcore, in the classic use of the word.<br /><br />It is called OZ as that is the nickname given to the Oswald Maximum Security State Penitentary. It focuses mainly on Emerald City, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda. Em City is home to many..Aryans, Muslims, gangstas, Latinos, Christians, Italians, Irish and more....so scuffles, death stares, dodgy dealings and shady agreements are never far away.<br /><br />I would say the main appeal of the show is due 

Tiene sentido normalizar el corpus, ya que el texto extraido incluye tags HTML, signos de puntuación y mayúsculas.

In [24]:
import nltk
import re
from os import path, getcwd

nltk_path = path.join(getcwd(), 'data')
nltk.data.path.append(nltk_path)
# nltk.download('stopwords', nltk_path)

wpt = nltk.WordPunctTokenizer()
stop_words = nltk.corpus.stopwords.words('english')


def normalize_document(doc):
    # lower case and remove special characters\whitespaces
    doc = re.sub('<br />', ' ', doc)
    doc = re.sub(r'[^a-zA-Z\s]', '', doc, re.I | re.A)
    doc = doc.lower()
    doc = doc.strip()
    # tokenize document
    tokens = wpt.tokenize(doc)
    # filter stopwords out of document
    filtered_tokens = [token for token in tokens if token not in stop_words]
    # re-create document from filtered tokens
    doc = ' '.join(filtered_tokens)
    return doc


normalize_corpus = np.vectorize(normalize_document)

In [25]:
df.review = normalize_corpus(df.review)

In [26]:
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences

max_words = 1500    # cogeremos las 1500 palabras más frecuentes
max_comment_length = 20 #las secuencias tendrán 20 palabras, el resto serán ceros

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(df.review)

sequences = tokenizer.texts_to_sequences(df.review)

word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))
max_words = len(word_index)

data = pad_sequences(sequences, maxlen=max_comment_length)

Found 162148 unique tokens.


In [27]:
word_index.keys()

dict_keys(['movie', 'film', 'one', 'like', 'good', 'even', 'would', 'time', 'really', 'see', 'story', 'well', 'much', 'get', 'bad', 'great', 'also', 'people', 'first', 'dont', 'movies', 'made', 'films', 'way', 'make', 'could', 'characters', 'think', 'watch', 'many', 'seen', 'two', 'character', 'never', 'love', 'acting', 'plot', 'best', 'know', 'little', 'show', 'life', 'ever', 'better', 'end', 'still', 'scene', 'say', 'man', 'scenes', 'something', 'go', 'im', 'back', 'watching', 'real', 'thing', 'doesnt', 'actors', 'didnt', 'years', 'though', 'funny', 'another', 'actually', 'makes', 'nothing', 'work', 'find', 'look', 'going', 'lot', 'new', 'every', 'old', 'us', 'part', 'director', 'cant', 'thats', 'things', 'want', 'cast', 'quite', 'pretty', 'seems', 'around', 'world', 'young', 'got', 'take', 'however', 'fact', 'enough', 'horror', 'big', 'thought', 'ive', 'give', 'may', 'long', 'without', 'right', 'isnt', 'music', 'saw', 'always', 'times', 'gets', 'original', 'series', 'comedy', 'come'

In [28]:
# tensorflow trabaja con variables numéricas,
# por lo que sustituimos los sentimientos
df.sentiment.replace({"positive": 1, "negative": 0}, inplace=True)

In [29]:
# Crea una partición de los datos dejando el 80% para entrenamiento y el 20% restante para test
# usando la función `train_test_split` de sklearn. 
#------------------------------------------------------------------------------

from sklearn.model_selection import train_test_split

d=df.values

x_train, x_test, y_train, y_test = train_test_split(data, df.sentiment, test_size=0.20, random_state=RANDOM_STATE, stratify = df.sentiment)

print("Training texts:", len(y_train))
print("Test texts:", len(y_test))

Training texts: 40000
Test texts: 10000


## 2) Estudio del efecto de distintas configuraciones de word embeddings para resolver la tara

Usa distintas configuraciones de word embeddigns y discute los resultados obtenidos.



In [30]:
# Fijamos el tamaño de los embedding a 50 dimensiones

embedding_dim = 50

a) Sin Embeddings pre-entrenados

In [33]:
# MODELO 1. SIN EMBEDDINGS PRE-ENTRENADOS 

from keras.models import Sequential
from keras.layers import Flatten, Dense, Embedding

model1 = Sequential()
# We specify the maximum input length to our Embedding layer
# so we can later flatten the embedded inputs


model1.add(Embedding(max_words, embedding_dim, input_length=max_comment_length))
# After the Embedding layer, our activations have shape `(max_words, max_comment_length, embedding_dim)`.

# We flatten the 3D tensor of embeddings into a 2D tensor of shape `(max_words, max_comment_length * embedding_dim)`

model1.add(Flatten())

# We add the classifier on top
model1.add(Dense(1, activation='sigmoid'))

model1.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model1.summary()

print("Fitting...")
history = model1.fit(x_train, y_train,
                    epochs=20,
                    batch_size=32,
                    validation_data=(x_test, y_test),
                    use_multiprocessing=True)

print("Evaluating...")
score1 = model1.evaluate(x_test, y_test,
                    use_multiprocessing=True)

print("Accuracy: %.2f%%" % (score1[1]*100))

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 20, 50)            8107400   
                                                                 
 flatten_3 (Flatten)         (None, 1000)              0         
                                                                 
 dense_3 (Dense)             (None, 1)                 1001      
                                                                 
Total params: 8,108,401
Trainable params: 8,108,401
Non-trainable params: 0
_________________________________________________________________
Fitting...
Epoch 1/20
1250/1250 [==============================] - 93s 74ms/step - loss: 0.4873 - accuracy: 0.7646 - val_loss: 0.4267 - val_accuracy: 0.8008
Epoch 2/20
1250/1250 [==============================] - 93s 75ms/step - loss: 0.3942 - accuracy: 0.8185 - val_loss: 0.4290 - val_accuracy: 0.7985
Epoch

Usando directamente la capa de embedding obtenemos un 72.51% de accuracy.

---------

b) Word embeddings pre-entrenados y congelados.

In [38]:
import numpy as np

embeddings_index = {}
with open('./glove.6B.50d.txt', encoding="UTF-8") as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs

print('Found %s word vectors.' % len(embeddings_index))

Found 400000 word vectors.


In [39]:
# Construimos nuestra matriz
embedding_matrix = np.zeros((max_words, embedding_dim))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if i < max_words:
        if embedding_vector is not None:
            # Words not found in embedding index will be all-zeros.
            embedding_matrix[i] = embedding_vector

In [41]:
# Definimos el modelo y cargamos las word embeddings de GloVe
from keras.models import Sequential
from keras.layers import Embedding, Flatten, Dense

model2 = Sequential()
model2.add(Embedding(max_words, embedding_dim, input_length=max_comment_length))
model2.add(Flatten())
model2.add(Dense(1, activation='sigmoid'))
model2.summary()

model2.layers[0].set_weights([embedding_matrix])
model2.layers[0].trainable = False  # El conjunto pre-entrenado no se verá afectado durante el entrenamiento

model2.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

print("Fitting...")
history = model2.fit(x_train, y_train,
                    epochs=20,
                    batch_size=32,
                    validation_data=(x_test, y_test))

print("Evaluating...")
score2 = model2.evaluate(x_test, y_test)

print("Accuracy: %.2f%%" % (score2[1]*100))

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_5 (Embedding)     (None, 20, 50)            8107400   
                                                                 
 flatten_5 (Flatten)         (None, 1000)              0         
                                                                 
 dense_5 (Dense)             (None, 1)                 1001      
                                                                 
Total params: 8,108,401
Trainable params: 8,108,401
Non-trainable params: 0
_________________________________________________________________
Fitting...
Epoch 1/20
1250/1250 [==============================] - 1s 751us/step - loss: 0.6071 - accuracy: 0.6680 - val_loss: 0.5737 - val_accuracy: 0.6959
Epoch 2/20
1250/1250 [==============================] - 1s 696us/step - loss: 0.5680 - accuracy: 0.7074 - val_loss: 0.5723 - val_accuracy: 0.6989
Epoch

El accuracy en este caso es del 69.85%, pero que el anterior aunque tardando mucho menos gracias al pre-entrenamiento.

C) Word embeddings pre-entrenados sin congelar.

In [42]:
from keras.models import Sequential
from keras.layers import Embedding, Flatten, Dense

model3 = Sequential()
model3.add(Embedding(max_words, embedding_dim, input_length=max_comment_length))
model3.add(Flatten())
model3.add(Dense(1, activation='sigmoid'))
model3.summary()

model3.layers[0].set_weights([embedding_matrix])
model3.layers[0].trainable = True

model3.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
history = model3.fit(x_train, y_train,
                    epochs=20,
                    batch_size=32,
                    validation_data=(x_test, y_test))

score3 = model3.evaluate(x_test, y_test)

print("Accuracy: %.2f%%" % (score3[1]*100))

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_6 (Embedding)     (None, 20, 50)            8107400   
                                                                 
 flatten_6 (Flatten)         (None, 1000)              0         
                                                                 
 dense_6 (Dense)             (None, 1)                 1001      
                                                                 
Total params: 8,108,401
Trainable params: 8,108,401
Non-trainable params: 0
_________________________________________________________________
Epoch 1/20
1250/1250 [==============================] - 91s 72ms/step - loss: 0.5446 - accuracy: 0.7136 - val_loss: 0.4647 - val_accuracy: 0.7793
Epoch 2/20
1250/1250 [==============================] - 91s 73ms/step - loss: 0.4253 - accuracy: 0.8012 - val_loss: 0.4392 - val_accuracy: 0.7930
Epoch 3/20
1250/

El accuracy en este ultimo caso es de 73.21%, el mejor resultado.

## 3) Análisis final

Analiza con detalle el mejor clasificador. Busca un ejemplo mal clasificado de cada clase, justifica el error ¿se te ocurre alguna forma de solucionarlo?

Compara los resultados obtenidos con y sin word embeddings


In [43]:
print("Sin word embeddings pre-entrenados")
print("Accuracy: %.2f%%" % (score1[1]*100))
print("Con word embeddings pre-entrenados congelados")
print("Accuracy: %.2f%%" % (score2[1]*100))
print("Con word embeddings pre-entrenados sin congelar")
print("Accuracy: %.2f%%" % (score3[1]*100))

Sin word embeddings pre-entrenados
Accuracy: 72.51%
Con word embeddings pre-entrenados congelados
Accuracy: 69.85%
Con word embeddings pre-entrenados sin congelar
Accuracy: 73.21%


TODO: EXPLICAR QUE LOS RESULTADOS SON PEORES EN EL SEGUNDO POR USARSE UNAS WORD EMBEDDINGS GENERALES SIN QUE PUEDAN REAJUSTARSE A UN CORPUS DE TEXTOS TAN GRANDE, COSA QUE SE ARREGLA EN EL TERCERO.

TODO: COMPARAR CON LOS RESULTADOS DEL OTRO NOTEBOOK